In [222]:
#pip install chardet

In [223]:
#pip install openpyxl

In [224]:
#pip install xlrd

## Analyse et exploration des données

In [225]:
import pandas as pd

In [226]:
import chardet

# Détecter l'encodage du fichier
with open("C:/Users/Ndiaye/Downloads/data_propre/gare/donnee de base/localisation-des-gares-et-haltes-ferroviaires-doccitanie.csv", "rb") as file:
    result = chardet.detect(file.read())
    print(result)


{'encoding': 'UTF-8-SIG', 'confidence': 1.0, 'language': ''}


In [227]:
import pandas as pd

df = pd.read_csv("C:\\Users\\Ndiaye\\Downloads\\data_propre\\gare\\donnee de base\\localisation-des-gares-et-haltes-ferroviaires-doccitanie.csv",sep=";", encoding='UTF-8-SIG')
df.head()


,objectid,id_rte500,nature,toponyme,type,codeuic,region,departement,date_maj,geo_shape,geo_point_2d
0,3,1456.0,Gare de voyageurs et de fret,Portet-Saint-Simon,gare avec accueil du public,87611400,Occitanie,31,2018-02-07T01:00:00+00:00,"{""coordinates"": [1.388634471956824, 43.5274872...","43.527487224320076, 1.388634471956824"
1,4,1457.0,Gare de voyageurs,Montrabé,halte ferroviaire,87611848,Occitanie,31,2018-02-07T01:00:00+00:00,"{""coordinates"": [1.5233678762803382, 43.645245...","43.645245668804705, 1.5233678762803382"
2,13,1584.0,Gare de voyageurs,Bourg-Madame,gare avec accueil du public,87784876,Occitanie,66,2018-02-07T01:00:00+00:00,"{""coordinates"": [1.9485651814585896, 42.432568...","42.43256868728393, 1.9485651814585896"
3,16,1595.0,Gare de voyageurs et de fret,Colomiers,gare avec accueil du public,87611467,Occitanie,31,2018-02-07T01:00:00+00:00,"{""coordinates"": [1.3340385138961044, 43.603269...","43.60326939335566, 1.3340385138961044"
4,29,1748.0,Gare de voyageurs,Grand-Combe-La Pise,gare avec accueil du public,87775254,Occitanie,30,2018-02-07T01:00:00+00:00,"{""coordinates"": [4.03327003378751, 44.20992003...","44.209920030108925, 4.03327003378751"


In [228]:
df_occitanie = df[df['region'] == 'Occitanie']
df_occitanie.head()

,objectid,id_rte500,nature,toponyme,type,codeuic,region,departement,date_maj,geo_shape,geo_point_2d
0,3,1456.0,Gare de voyageurs et de fret,Portet-Saint-Simon,gare avec accueil du public,87611400,Occitanie,31,2018-02-07T01:00:00+00:00,"{""coordinates"": [1.388634471956824, 43.5274872...","43.527487224320076, 1.388634471956824"
1,4,1457.0,Gare de voyageurs,Montrabé,halte ferroviaire,87611848,Occitanie,31,2018-02-07T01:00:00+00:00,"{""coordinates"": [1.5233678762803382, 43.645245...","43.645245668804705, 1.5233678762803382"
2,13,1584.0,Gare de voyageurs,Bourg-Madame,gare avec accueil du public,87784876,Occitanie,66,2018-02-07T01:00:00+00:00,"{""coordinates"": [1.9485651814585896, 42.432568...","42.43256868728393, 1.9485651814585896"
3,16,1595.0,Gare de voyageurs et de fret,Colomiers,gare avec accueil du public,87611467,Occitanie,31,2018-02-07T01:00:00+00:00,"{""coordinates"": [1.3340385138961044, 43.603269...","43.60326939335566, 1.3340385138961044"
4,29,1748.0,Gare de voyageurs,Grand-Combe-La Pise,gare avec accueil du public,87775254,Occitanie,30,2018-02-07T01:00:00+00:00,"{""coordinates"": [4.03327003378751, 44.20992003...","44.209920030108925, 4.03327003378751"


# Calcul des distances entre les gares pour trouver le taux de co2 emis 

In [229]:
from geopy.distance import geodesic

# Séparation de la colonne geo_point_2d en latitude et longitude
df[['latitude', 'longitude']] = df['geo_point_2d'].str.split(', ', expand=True).astype(float)

# Fonction pour calculer la distance entre deux gares
def calculer_distance(gare1, gare2):
    coords1 = (gare1['latitude'], gare1['longitude'])  # (latitude, longitude)
    coords2 = (gare2['latitude'], gare2['longitude'])
    return geodesic(coords1, coords2).kilometers  # Distance en kilomètres



In [230]:
# Calcul la distance entre deux gares
gare_A = df[df['toponyme'] == 'Toulouse-Matabiau'].iloc[0]  # Gare A
gare_B = df[df['toponyme'] == 'Perpignan'].iloc[0]  # Gare B
distance = calculer_distance(gare_A, gare_B)
print(f"Distance entre {gare_A['toponyme']} et {gare_B['toponyme']} : {distance:.2f} km")

Distance entre Toulouse-Matabiau et Perpignan : 154.28 km


In [231]:
# Créer une matrice vide
n = len(df)
distances = pd.DataFrame(index=df['toponyme'], columns=df['toponyme'])

# Remplir la matrice avec les distances
for i, gare1 in df.iterrows():
    for j, gare2 in df.iterrows():
        distances.at[gare1['toponyme'], gare2['toponyme']] = calculer_distance(gare1, gare2)
distances

toponyme,Portet-Saint-Simon,Montrabé,Bourg-Madame,Colomiers,Grand-Combe-La Pise,Souillac,Le Monastier,Olette-Canaveilles-les-Bains,Pamiers,Quillan,...,Sauto,Thuès-Carença,Le Vernet-d'Ariège,Lacourtensourt,Beaucaire,Assier,Roquemaure,Agen,Nîmes Pont du Gard,Marseille Saint-Charles
toponyme,,,,,,,,,,,,,,,,,,,,,
Portet-Saint-Simon,0.0,17.016994,129.927977,9.50505,225.688336,152.870516,185.007236,129.875642,49.424775,97.096731,...,129.707755,130.787191,41.558474,15.277517,264.571222,133.154418,278.559154,97.623024,253.498193,324.277844
Montrabé,17.016994,0.0,139.104338,15.976742,211.083602,139.732275,168.508878,135.642507,59.329241,101.007272,...,136.743989,137.140912,51.006195,9.069809,252.297395,117.686904,265.187992,95.736117,241.106631,314.414829
Bourg-Madame,129.927977,139.104338,0.0,139.369636,259.954471,277.115638,253.690979,29.883946,80.534593,52.600065,...,19.195962,25.046362,88.610713,143.547694,267.162952,249.188763,291.386977,224.755227,258.809297,296.639697
Colomiers,9.50505,15.976742,139.369636,0.0,227.056799,144.712124,183.736185,139.326295,58.896365,106.335984,...,139.209258,140.268875,51.059866,9.310075,267.952881,126.521861,281.0812,88.322151,256.790866,329.228714
Grand-Combe-La Pise,225.688336,211.083602,259.954471,227.056799,0.0,217.925477,70.587135,232.748612,229.483086,210.787282,...,242.670039,237.737254,225.65691,218.883388,66.907236,179.8305,61.895677,272.702025,57.854285,148.140805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Assier,133.154418,117.686904,249.188763,126.521861,179.8305,40.665231,111.336381,237.783227,174.3545,201.669919,...,242.067267,240.719144,166.181411,118.088274,242.139464,0.0,241.719785,112.079344,231.194755,320.282648
Roquemaure,278.559154,265.187992,291.386977,281.0812,61.895677,279.632163,131.772299,262.22011,275.019484,247.11092,...,272.777555,267.281591,272.464659,273.497175,29.29207,241.719785,0.0,332.796104,33.564485,96.356432
Agen,97.623024,95.736117,224.755227,88.322151,272.702025,102.271039,212.410049,227.290268,145.680646,194.656647,...,226.535259,227.975533,138.424307,87.828804,326.000418,112.079344,332.796104,0.0,314.581911,396.27596


In [232]:
# Sauvegarde la matrice dans un fichier CSV
distances.to_csv('distances_gares.csv')

In [233]:
# une liste pour stocker les résultats
resultats = []

# distances pour toutes les paires de gares
for i, gare1 in df.iterrows():
    for j, gare2 in df.iterrows():
        if i < j:  # Pour éviter les doublons (A-B et B-A) et les distances nulles (A-A)
            distance = calculer_distance(gare1, gare2)
            resultats.append({
                'Gare A': gare1['toponyme'],
                'Gare B': gare2['toponyme'],
                'Distance (km)': distance
            })
# Convertir la liste en DataFrame
resultats_df = pd.DataFrame(resultats)

In [234]:
resultats_df

,Gare A,Gare B,Distance (km)
0,Portet-Saint-Simon,Montrabé,17.016994
1,Portet-Saint-Simon,Bourg-Madame,129.927977
2,Portet-Saint-Simon,Colomiers,9.505050
3,Portet-Saint-Simon,Grand-Combe-La Pise,225.688336
4,Portet-Saint-Simon,Souillac,152.870516
...,...,...,...
43655,Roquemaure,Nîmes Pont du Gard,33.564485
43656,Roquemaure,Marseille Saint-Charles,96.356432
43657,Agen,Nîmes Pont du Gard,314.581911
43658,Agen,Marseille Saint-Charles,396.275960


In [235]:
def rechercher_gare(df, nom_gare):
    """
    Recherche une gare par son nom (toponyme) et retourne la première occurrence.
    """
    resultats = df[df['toponyme'].str.contains(nom_gare, case=False, na=False)]
    if not resultats.empty:
        return resultats.iloc[0]  # Retourne la première gare trouvée
    else:
        return None  # Si aucune gare n'est trouvée

In [236]:
# # Rechercher des gares et calcule de la distance
# nom_gare_A = input("Entrez le nom de la première gare : ")  # Exemple : "Toulouse-Matabiau"
# nom_gare_B = input("Entrez le nom de la deuxième gare : ")  # Exemple : "Perpignan"

# # Rechercher les gares
# gare_A = rechercher_gare(df, nom_gare_A)
# gare_B = rechercher_gare(df, nom_gare_B)

# # Vérification si les gares ont été trouvées
# if gare_A is not None and gare_B is not None:
#     distance = calculer_distance(gare_A, gare_B)
#     print(f"Distance entre {gare_A['toponyme']} et {gare_B['toponyme']} : {distance:.2f} km")
# else:
#     print("Une ou plusieurs gares n'ont pas été trouvées.")

In [237]:
df.isnull().sum()  # Affiche le nombre de valeurs manquantes par colonne

objectid         0
id_rte500       21
nature          21
toponyme         0
type            17
codeuic          0
region           0
departement      0
date_maj         0
geo_shape        0
geo_point_2d     0
latitude         0
longitude        0
dtype: int64

In [238]:
import pandas as pd
from geopy.distance import geodesic

# Charger le fichier Excel
df2 = pd.read_csv("C:\\Users\\Ndiaye\\Downloads\\data_propre\\gare\\donnee de base\\localisation-des-gares-et-haltes-ferroviaires-doccitanie.csv",sep=";", encoding='UTF-8-SIG')



In [239]:
# Séparation de la colonne geo_point_2d en latitude et longitude
df2[['latitude', 'longitude']] = df2['geo_point_2d'].str.split(', ', expand=True).astype(float)

# Facteurs d'émission (en kg CO₂/km)
facteur_train = 0.024  # Train
facteur_voiture = 0.2  # Voiture



In [240]:
# Fonction pour calculer la distance entre deux gares
def calculer_distance(gare11, gare22):
    coords1 = (gare11['latitude'], gare11['longitude'])  # (latitude, longitude)
    coords2 = (gare22['latitude'], gare22['longitude'])
    return geodesic(coords1, coords2).kilometers  # Distance en kilomètres

# Fonction pour calculer les émissions de CO₂
def calculer_emissions(distance, facteur, passagers=1):
    return distance * facteur * passagers



In [241]:
# les émissions pour un trajet
gare_A = df2[df2['toponyme'] == 'Toulouse-Matabiau'].iloc[0]  # Gare A
gare_B = df2[df2['toponyme'] == 'Montpellier-Saint-Roch'].iloc[0]  # Gare B
distance = calculer_distance(gare_A, gare_B)

emissions_train = calculer_emissions(distance, facteur_train)
emissions_voiture = calculer_emissions(distance, facteur_voiture)
economies = emissions_voiture - emissions_train

print(f"Distance entre {gare_A['toponyme']} et {gare_B['toponyme']} : {distance:.2f} km")
print(f"Émissions CO₂ (train) : {emissions_train:.2f} kg CO₂")
print(f"Émissions CO₂ (voiture) : {emissions_voiture:.2f} kg CO₂")
print(f"Économies CO₂ : {economies:.2f} kg CO₂")

Distance entre Toulouse-Matabiau et Montpellier-Saint-Roch : 195.93 km
Émissions CO₂ (train) : 4.70 kg CO₂
Émissions CO₂ (voiture) : 39.19 kg CO₂
Économies CO₂ : 34.48 kg CO₂


In [242]:
# liste pour stocker les résultats
resultats = []

# Calcule l'émissions pour toutes les paires de gares
for i, gare11 in df2.iterrows():
    for j, gare22 in df2.iterrows():
        if i < j:  # Pour éviter les doublons (A-B et B-A) et les distances nulles (A-A)
            distance = calculer_distance(gare11, gare22)
            emissions_train = calculer_emissions(distance, facteur_train)
            emissions_voiture = calculer_emissions(distance, facteur_voiture)
            economies = emissions_voiture - emissions_train
            resultats.append({
                'Gare A': gare11['toponyme'],
                'Gare B': gare22['toponyme'],
                'Distance (km)': distance,
                'Émissions CO₂ (train)': emissions_train,
                'Émissions CO₂ (voiture)': emissions_voiture,
                'Économies CO₂': economies
            })

# Convertir la liste en DataFrame
resultats_df2 = pd.DataFrame(resultats)

# Sauvegarder les résultats dans un fichier Excel
#resultats_df2.to_excel('emissions_co2_gares.xlsx', index=False)


In [243]:
resultats_df2.to_csv('emissions_co2_gares.csv', index=False)


In [244]:
print("Résultats sauvegardés dans emissions_co2_gares.xlsx")

Résultats sauvegardés dans emissions_co2_gares.xlsx


In [245]:
df3 = pd.read_csv("C:\\Users\\Ndiaye\\Downloads\\data_propre\\gare\\donnee de base\\localisation-des-gares-et-haltes-ferroviaires-doccitanie.csv",sep=";", encoding='UTF-8-SIG')


In [246]:
doublons1 = df3[df3.duplicated(subset=['toponyme'], keep=False)]  # Affiche tous les doublons

# Afficher les doublons
doublons1

,objectid,id_rte500,nature,toponyme,type,codeuic,region,departement,date_maj,geo_shape,geo_point_2d


In [247]:
# Supprimer les doublons et garder la première occurrence
df_clean = df3.drop_duplicates(subset=['toponyme'], keep='first')

# Sauvegarder le fichier nettoyé
df_clean.to_csv('gares_cleaned.csv', index=False)


In [248]:
# Extraire latitude et longitude à partir de la colonne 'geo_point_2d'
df3[['latitude', 'longitude']] = df3['geo_point_2d'].str.split(', ', expand=True)

# Convertir les colonnes latitude et longitude en float
df3['latitude'] = df3['latitude'].astype(float)
df3['longitude'] = df3['longitude'].astype(float)

df3

,objectid,id_rte500,nature,toponyme,type,codeuic,region,departement,date_maj,geo_shape,geo_point_2d,latitude,longitude
0,3,1456.0,Gare de voyageurs et de fret,Portet-Saint-Simon,gare avec accueil du public,87611400,Occitanie,31,2018-02-07T01:00:00+00:00,"{""coordinates"": [1.388634471956824, 43.5274872...","43.527487224320076, 1.388634471956824",43.527487,1.388634
1,4,1457.0,Gare de voyageurs,Montrabé,halte ferroviaire,87611848,Occitanie,31,2018-02-07T01:00:00+00:00,"{""coordinates"": [1.5233678762803382, 43.645245...","43.645245668804705, 1.5233678762803382",43.645246,1.523368
2,13,1584.0,Gare de voyageurs,Bourg-Madame,gare avec accueil du public,87784876,Occitanie,66,2018-02-07T01:00:00+00:00,"{""coordinates"": [1.9485651814585896, 42.432568...","42.43256868728393, 1.9485651814585896",42.432569,1.948565
3,16,1595.0,Gare de voyageurs et de fret,Colomiers,gare avec accueil du public,87611467,Occitanie,31,2018-02-07T01:00:00+00:00,"{""coordinates"": [1.3340385138961044, 43.603269...","43.60326939335566, 1.3340385138961044",43.603269,1.334039
4,29,1748.0,Gare de voyageurs,Grand-Combe-La Pise,gare avec accueil du public,87775254,Occitanie,30,2018-02-07T01:00:00+00:00,"{""coordinates"": [4.03327003378751, 44.20992003...","44.209920030108925, 4.03327003378751",44.209920,4.033270
...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,299,4798.0,Gare de voyageurs,Assier,gare avec accueil du public,87613075,Occitanie,46,2018-02-07T01:00:00+00:00,"{""coordinates"": [1.8693908513648663, 44.674681...","44.674681446354775, 1.8693908513648663",44.674681,1.869391
292,320,NaN,NaN,Roquemaure,NaN,87745059,Occitanie,30,2024-12-20T01:00:00+00:00,"{""coordinates"": [4.7738600000173, 44.049400000...","44.04940000015025, 4.7738600000173",44.049400,4.773860
293,326,NaN,NaN,Agen,gare avec accueil du public,87586008,Nouvelle-Aquitaine,47,2025-01-06T01:00:00+00:00,"{""coordinates"": [0.6210055931407953, 44.208347...","44.20834730436371, 0.6210055931407953",44.208347,0.621006
294,737,NaN,NaN,Nîmes Pont du Gard,NaN,87703975,Occitanie,30,2025-01-15T01:00:00+00:00,"{""coordinates"": [4.506744961675008, 43.8170413...","43.81704130744609, 4.506744961675008",43.817041,4.506745


In [249]:
df3.to_csv('gares_traiter.csv', index=False)


In [250]:
df4 = pd.read_csv("C:\\Users\\Ndiaye\\Downloads\\data_propre\\gare\\donnee de base\\distance.csv")

In [251]:
df4

,InputID,TargetID,Distance
0,Portet-Saint-Simon,Montrabé,17016.994284
1,Portet-Saint-Simon,Bourg-Madame,129927.977064
2,Portet-Saint-Simon,Colomiers,9505.050181
3,Portet-Saint-Simon,Grand-Combe-La Pise,225688.336279
4,Portet-Saint-Simon,Souillac,152870.516410
...,...,...,...
87315,Marseille Saint-Charles,Beaucaire,81233.573891
87316,Marseille Saint-Charles,Assier,320282.647611
87317,Marseille Saint-Charles,Roquemaure,96356.431666
87318,Marseille Saint-Charles,Agen,396275.960272


In [252]:
df4['Distance_km'] = df4['Distance'] / 1000  # Conversion de mètres à kilomètres


In [253]:
df4

,InputID,TargetID,Distance,Distance_km
0,Portet-Saint-Simon,Montrabé,17016.994284,17.016994
1,Portet-Saint-Simon,Bourg-Madame,129927.977064,129.927977
2,Portet-Saint-Simon,Colomiers,9505.050181,9.505050
3,Portet-Saint-Simon,Grand-Combe-La Pise,225688.336279,225.688336
4,Portet-Saint-Simon,Souillac,152870.516410,152.870516
...,...,...,...,...
87315,Marseille Saint-Charles,Beaucaire,81233.573891,81.233574
87316,Marseille Saint-Charles,Assier,320282.647611,320.282648
87317,Marseille Saint-Charles,Roquemaure,96356.431666,96.356432
87318,Marseille Saint-Charles,Agen,396275.960272,396.275960


In [254]:
# Facteurs d'émission (par exemple)
facteur_train = 0.03  # gCO2/km/passager (pour un train)
facteur_voiture = 0.2  # gCO2/km (pour une voiture)

# Ajouter les colonnes pour les émissions
df4['emissions_train'] = df4['Distance_km'] * facteur_train / 1000  # En kgCO2
df4['emissions_voiture'] = df4['Distance_km'] * facteur_voiture / 1000  # En kgCO2
df4['economies'] = df4['emissions_voiture'] - df4['emissions_train']  # Economies de CO2

# Afficher les premières lignes du DataFrame
df4.head()

# Sauvegarder les résultats dans un nouveau fichier CSV
#rdf.to_csv('emissions_co2_gares.csv', index=False)

,InputID,TargetID,Distance,Distance_km,emissions_train,emissions_voiture,economies
0,Portet-Saint-Simon,Montrabé,17016.994284,17.016994,0.000511,0.003403,0.002893
1,Portet-Saint-Simon,Bourg-Madame,129927.977064,129.927977,0.003898,0.025986,0.022088
2,Portet-Saint-Simon,Colomiers,9505.050181,9.505050,0.000285,0.001901,0.001616
3,Portet-Saint-Simon,Grand-Combe-La Pise,225688.336279,225.688336,0.006771,0.045138,0.038367
4,Portet-Saint-Simon,Souillac,152870.516410,152.870516,0.004586,0.030574,0.025988


In [255]:
# Sauvegarder les résultats dans un nouveau fichier CSV
df4.to_csv('emissions_co2_gares.csv', index=False)

In [256]:
# Renommer les colonnes
df4.rename(columns={'InputID': 'Gare_Origine', 'TargetID': 'Gare_Destination'}, inplace=True)
df4

,Gare_Origine,Gare_Destination,Distance,Distance_km,emissions_train,emissions_voiture,economies
0,Portet-Saint-Simon,Montrabé,17016.994284,17.016994,0.000511,0.003403,0.002893
1,Portet-Saint-Simon,Bourg-Madame,129927.977064,129.927977,0.003898,0.025986,0.022088
2,Portet-Saint-Simon,Colomiers,9505.050181,9.505050,0.000285,0.001901,0.001616
3,Portet-Saint-Simon,Grand-Combe-La Pise,225688.336279,225.688336,0.006771,0.045138,0.038367
4,Portet-Saint-Simon,Souillac,152870.516410,152.870516,0.004586,0.030574,0.025988
...,...,...,...,...,...,...,...
87315,Marseille Saint-Charles,Beaucaire,81233.573891,81.233574,0.002437,0.016247,0.013810
87316,Marseille Saint-Charles,Assier,320282.647611,320.282648,0.009608,0.064057,0.054448
87317,Marseille Saint-Charles,Roquemaure,96356.431666,96.356432,0.002891,0.019271,0.016381
87318,Marseille Saint-Charles,Agen,396275.960272,396.275960,0.011888,0.079255,0.067367


In [257]:
# Sauvegarder les résultats dans un nouveau fichier CSV
df4.to_csv('gare_distance_co2.csv', index=False)


In [258]:
df5 = pd.read_csv("C:\\Users\\Ndiaye\\Downloads\\data_propre\\gare\\donnee de base\\ligne_ferro.csv")

In [259]:
df3.head()

,objectid,id_rte500,nature,toponyme,type,codeuic,region,departement,date_maj,geo_shape,geo_point_2d,latitude,longitude
0,3,1456.0,Gare de voyageurs et de fret,Portet-Saint-Simon,gare avec accueil du public,87611400,Occitanie,31,2018-02-07T01:00:00+00:00,"{""coordinates"": [1.388634471956824, 43.5274872...","43.527487224320076, 1.388634471956824",43.527487,1.388634
1,4,1457.0,Gare de voyageurs,Montrabé,halte ferroviaire,87611848,Occitanie,31,2018-02-07T01:00:00+00:00,"{""coordinates"": [1.5233678762803382, 43.645245...","43.645245668804705, 1.5233678762803382",43.645246,1.523368
2,13,1584.0,Gare de voyageurs,Bourg-Madame,gare avec accueil du public,87784876,Occitanie,66,2018-02-07T01:00:00+00:00,"{""coordinates"": [1.9485651814585896, 42.432568...","42.43256868728393, 1.9485651814585896",42.432569,1.948565
3,16,1595.0,Gare de voyageurs et de fret,Colomiers,gare avec accueil du public,87611467,Occitanie,31,2018-02-07T01:00:00+00:00,"{""coordinates"": [1.3340385138961044, 43.603269...","43.60326939335566, 1.3340385138961044",43.603269,1.334039
4,29,1748.0,Gare de voyageurs,Grand-Combe-La Pise,gare avec accueil du public,87775254,Occitanie,30,2018-02-07T01:00:00+00:00,"{""coordinates"": [4.03327003378751, 44.20992003...","44.209920030108925, 4.03327003378751",44.209920,4.033270


In [260]:
#pip install openrouteservice


In [261]:
# Extraire la distance du trajet
distance = routes['features'][0]['properties']['segments'][0]['distance'] / 1000  # en kilomètres
print(f"La distance réelle entre les deux  est de : {distance} km")


La distance réelle entre les deux  est de : 207.1796 km


In [262]:
import openrouteservice
import requests
import csv

In [263]:
df_gares = pd.read_csv("C:\\Users\\Ndiaye\\Downloads\\data_propre\\gare\\donnee de base\\localisation-des-gares-et-haltes-ferroviaires-doccitanie.csv",sep=";", encoding='UTF-8-SIG')

In [264]:
# Extraire latitude et longitude à partir de la colonne 'geo_point_2d'
df_gares[['latitude', 'longitude']] = df_gares['geo_point_2d'].str.split(', ', expand=True)

# Convertir les colonnes latitude et longitude en float
df_gares['latitude'] = df_gares['latitude'].astype(float)
df_gares['longitude'] = df_gares['longitude'].astype(float)
df_gares.head()

,objectid,id_rte500,nature,toponyme,type,codeuic,region,departement,date_maj,geo_shape,geo_point_2d,latitude,longitude
0,3,1456.0,Gare de voyageurs et de fret,Portet-Saint-Simon,gare avec accueil du public,87611400,Occitanie,31,2018-02-07T01:00:00+00:00,"{""coordinates"": [1.388634471956824, 43.5274872...","43.527487224320076, 1.388634471956824",43.527487,1.388634
1,4,1457.0,Gare de voyageurs,Montrabé,halte ferroviaire,87611848,Occitanie,31,2018-02-07T01:00:00+00:00,"{""coordinates"": [1.5233678762803382, 43.645245...","43.645245668804705, 1.5233678762803382",43.645246,1.523368
2,13,1584.0,Gare de voyageurs,Bourg-Madame,gare avec accueil du public,87784876,Occitanie,66,2018-02-07T01:00:00+00:00,"{""coordinates"": [1.9485651814585896, 42.432568...","42.43256868728393, 1.9485651814585896",42.432569,1.948565
3,16,1595.0,Gare de voyageurs et de fret,Colomiers,gare avec accueil du public,87611467,Occitanie,31,2018-02-07T01:00:00+00:00,"{""coordinates"": [1.3340385138961044, 43.603269...","43.60326939335566, 1.3340385138961044",43.603269,1.334039
4,29,1748.0,Gare de voyageurs,Grand-Combe-La Pise,gare avec accueil du public,87775254,Occitanie,30,2018-02-07T01:00:00+00:00,"{""coordinates"": [4.03327003378751, 44.20992003...","44.209920030108925, 4.03327003378751",44.209920,4.033270


In [265]:
df_gares['toponyme'] = df_gares['toponyme'].str.strip().str.upper()


In [266]:
#coordo = pd.read_csv()
df_parcours = pd.read_csv("C:\\Users\\Ndiaye\\Downloads\\traitement\\cordo.csv")
df_parcours

,Parcours,Départ,Arrivée,Nombre de voyages,Poids de voyages,Nombre d'abonnements
0,ALBI - TOULOUSE MATABIAU,ALBI,TOULOUSE MATABIAU,2330,"0,0401661811098278",0
1,MONTPELLIER SAINT-ROCH - NIMES,MONTPELLIER SAINT-ROCH,NIMES,2308,"0,0397869296143702",0
2,MONTAUBAN - TOULOUSE MATABIAU,MONTAUBAN,TOULOUSE MATABIAU,2169,"0,0373907497112517",0
3,CARCASSONNE - TOULOUSE MATABIAU,CARCASSONNE,TOULOUSE MATABIAU,1862,"0,0320984674791843",0
4,BEZIERS - MONTPELLIER SAINT-ROCH,BEZIERS,MONTPELLIER SAINT-ROCH,1528,"0,0263407402299643",0
...,...,...,...,...,...,...
1016,SAINT-CHÉLY-D'APCHER - ST ROME DE CERNON,SAINT-CHÉLY-D'APCHER,ST ROME DE CERNON,1,"0,0000172387043389819",0
1017,SAINT-DENIS-PRES-MARTEL - ST CHRISTOPHE,SAINT-DENIS-PRES-MARTEL,ST CHRISTOPHE,1,"0,0000172387043389819",0
1018,SAINT-SULPICE - TANUS,SAINT-SULPICE,TANUS,1,"0,0000172387043389819",0
1019,TARASCON SUR ARIEGE - VERNET-D'ARIEGE,TARASCON SUR ARIEGE,VERNET-D'ARIEGE,1,"0,0000172387043389819",0


In [267]:
line = pd.read_csv("C:\\Users\\Ndiaye\\Downloads\\traitement\\ligne_ferro.csv")
line

,objectid,ref_tronc,nom_tronc,ligne1,ligne2,ligne3,nb_voie,type_voie,uic,st_length_s
0,3,BS-AGD,Béziers-Agde,Ligne littorale,NaN,NaN,2,ligne électrifiée,3,21142.080668
1,17,TES-AL,Tessonnières-Albi,Toulouse-Rodez,NaN,NaN,1,ligne non électrifiée,7AV,16363.648007
2,16,TS-LD,Tarbes-Lourdes,Toulouse-Pau,NaN,NaN,2,ligne électrifiée,5,21201.811797
3,39,SST-TES,Saint Sulpice-Tessonnières,Toulouse-Rodez,Toulouse-Aurillac,NaN,1,ligne non électrifiée,6,27362.941546
4,57,SVC-MAU,Séverac-Millau,Rodez-Millau,NaN,NaN,1,ligne électrifiée,9AV,30968.016792
...,...,...,...,...,...,...,...,...,...,...
64,38,SYH-CLE,Saint Chély-Clermont Ferrand,Béziers-Clermont Ferrand,NaN,NaN,1,ligne électrifiée,9AV,163800.550431
65,43,VVB-FRM,Villefranche-Font Romeu,Train Jaune,NaN,NaN,1,ligne électrifiée,9AV,35273.064270
66,47,PPN-NA,Perpignan-Narbonne,Ligne littorale,NaN,NaN,2,ligne électrifiée,4,65012.974863
67,55,ALE-GNC,Alès-Génolhac,Nîmes-Clermont Ferrand,NaN,NaN,1,ligne non électrifiée,9AV,34309.354765


In [268]:
df_gares.to_csv('gare_Maj.csv', index=False)

In [269]:
df_parcours['Départ'] = df_parcours['Départ'].str.strip().str.upper()
df_parcours['Arrivée'] = df_parcours['Arrivée'].str.strip().str.upper()
df_gares['toponyme'] = df_gares['toponyme'].str.strip().str.upper()



In [270]:
# jointure pour avoir les coodonnée longitude et latitude
df_parcours = pd.merge(df_parcours, df_gares[['toponyme', 'latitude', 'longitude']],
                        how='left', left_on='Départ', right_on='toponyme', 
                        suffixes=('_depart', '_gare'))

df_parcours = pd.merge(df_parcours, df_gares[['toponyme', 'latitude', 'longitude']],
                        how='left', left_on='Arrivée', right_on='toponyme', 
                        suffixes=('_depart', '_arrivee'))


In [ ]:
# Sauvegarder le fichier avec les coordonnées
df_parcours.to_csv('coordonnées.csv', index=False)
print("Les coordonnées ont été ajoutées et sauvegardées dans le fichier 'parcours_avec_coordonnees.csv'.")

Les coordonnées ont été ajoutées et sauvegardées dans le fichier 'parcours_avec_coordonnees.csv'.


In [ ]:
 # Charger les données de parcours (ex: départ, arrivée, nombre de voyages)
#df_parcours = pd.read_csv("C:\\Users\\Ndiaye\\Downloads\\voyages.csv", sep=";")
df_parcours = pd.read_csv("C:\\Users\\Ndiaye\\Downloads\\voyages.csv", encoding="Windows-1252", sep=";")

df_parcours

## Recherche des coordonnées via des API. Mais cette partie n'a pas abouti pour moi

In [ ]:

import requests

# Remplace par ta clé API OpenCage
API_KEY = '7beaca574d844a36a8c847481428c41e'

# Fonction pour géocoder une gare
def geocoder_gare(nom_gare):
    url = 'https://api.opencagedata.com/geocode/v1/json'
    params = {
        'q': nom_gare,  # Nom de la gare à rechercher
        'key': API_KEY,
        'language': 'fr',  # Langue des résultats
        'limit': 1  # Limite à 1 résultat
    }

    # Faire la requête à l'API
    response = requests.get(url, params=params)

    # Vérifier si la requête a réussi
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            # Extraire les coordonnées
            lat = data['results'][0]['geometry']['lat']
            lon = data['results'][0]['geometry']['lng']
            return lat, lon
        else:
            print(f"Aucun résultat trouvé pour {nom_gare}")
            return None
    else:
        print(f"Erreur : {response.status_code}")
        return None

# Exemple : Géocoder une gare
nom_gare = "Toulouse-Matabiau"
coords = geocoder_gare(nom_gare)
if coords:
    print(f"Coordonnées de {nom_gare} : {coords}")

In [ ]:
import openrouteservice

# Remplace par ta clé API OpenRouteService
api_key = '5b3ce3597851110001cf6248332856e9064749b294bd32932862c012'
client = openrouteservice.Client(key=api_key)

# Exemple de coordonnées de départ et d'arrivée (Latitude, Longitude)
coords_A = (2.8796272182832188, 42.696086371190745)  # Exemple : Gare de Perpignan
coords_B = (1.4540057050370228, 43.61214504394223)  # Exemple : Gare de Toulouse-Matabiau

# Calcul de l'itinéraire entre les gares
routes = client.directions(
    coordinates=[coords_A, coords_B],
    profile='driving-car',  # Pour les trajets en voiture (remplace par 'cycling' ou 'foot' si nécessaire)
    format='geojson'
)



# Vu que on ne cherche pas les trajets directs entre les gares et qu'on cherchais les itinéraire des trains, on a eu recours aux api(googleMaps) pour trouver les distances exactes.

In [ ]:
import requests
import pandas as pd

# Remplace par ta clé API OpenCage
API_KEY = '7beaca574d844a36a8c847481428c41e'

# Fonction pour géocoder une gare
def geocoder_gare(nom_gare):
    url = 'https://api.opencagedata.com/geocode/v1/json'
    params = {
        'q': nom_gare,  # Nom de la gare à rechercher
        'key': API_KEY,
        'language': 'fr',  # Langue des résultats
        'limit': 1  # Limite à 1 résultat
    }

    # Faire la requête à l'API
    response = requests.get(url, params=params)

    # Vérifier si la requête a réussi
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            # Extraire les coordonnées
            lat = data['results'][0]['geometry']['lat']
            lon = data['results'][0]['geometry']['lng']
            return lat, lon
        else:
            print(f"Aucun résultat trouvé pour {nom_gare}")
            return None
    else:
        print(f"Erreur : {response.status_code}")
        return None

# Charger votre fichier CSV avec les informations des gares
df_parcours = pd.read_csv("C:\\Users\\Ndiaye\\Downloads\\traitement\\cordo.csv")

# Ajouter les colonnes pour les coordonnées de départ et d'arrivée
df['latitude_depart'] = None
df['longitude_depart'] = None
df['latitude_arrivee'] = None
df['longitude_arrivee'] = None

# Géocoder les gares de départ et d'arrivée
for index, row in df.iterrows():
    gare_depart = row['Départ']
    gare_arrivee = row['Arrivée']
    
    # Géocoder la gare de départ
    coords_depart = geocoder_gare(gare_depart)
    if coords_depart:
        df.at[index, 'latitude_depart'] = coords_depart[0]
        df.at[index, 'longitude_depart'] = coords_depart[1]
    
    # Géocoder la gare d'arrivée
    coords_arrivee = geocoder_gare(gare_arrivee)
    if coords_arrivee:
        df.at[index, 'latitude_arrivee'] = coords_arrivee[0]
        df.at[index, 'longitude_arrivee'] = coords_arrivee[1]



In [ ]:
# Sauvegarder le fichier avec les nouvelles coordonnées
df.to_csv('gares_avec_coordonnees.csv', index=False)

print("Les coordonnées ont été ajoutées et sauvegardées dans le fichier 'gares_avec_coordonnees.csv'.")


In [ ]:
import openrouteservice

# Remplace par ta clé API OpenRouteService
api_key = '5b3ce3597851110001cf6248332856e9064749b294bd32932862c012'
client = openrouteservice.Client(key=api_key)

# Exemple de coordonnées de départ et d'arrivée (Latitude, Longitude)
coords_A = (2.8796272182832188, 42.696086371190745)  # Exemple : Gare de Perpignan
coords_B = (1.4540057050370228, 43.61214504394223)  # Exemple : Gare de Toulouse-Matabiau

# Calcul de l'itinéraire entre les gares
routes = client.directions(
    coordinates=[coords_A, coords_B],
    profile='driving-car',  # Pour les trajets en voiture (remplace par 'cycling' ou 'foot' si nécessaire)
    format='geojson'
)

